# WRF perturbations - difference in wind relaxation schemes

In [1]:
%cd git/wrf_lrf_les/

/home/561/tr2908/git/wrf_lrf_les


In [2]:
import sys
sys.path.append('analysis/')

import modules.wrf_perturbation as wp
from dask.distributed import Client
import matplotlib.pyplot as plt
from importlib import reload
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import xarray
import dask

print("Report last updated at " + str(datetime.datetime.utcnow()) + ' UTC.')

Report last updated at 2024-06-12 12:58:23.067387 UTC.


In [3]:
client = Client()
print(client)

<Client: 'tcp://127.0.0.1:43227' processes=7 threads=28, memory=125.18 GiB>


2024-06-12 23:18:56,004 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-06-12 23:19:07,016 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-06-12 23:19:13,081 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:39377' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('concatenate-149be54c170da2cc4b7551d057b78f75', 63, 0), ('concatenate-149be54c170da2cc4b7551d057b78f75', 17, 0), 'original-open_dataset-T-4b705874687ee152721485a646e2677b', ('concatenate-149be54c170da2cc4b7551d057b78f75', 76, 0), ('open_dataset-T-concatenate-149be54c170da2cc4b7551d057b78f75', 23, 0), ('concatenate-149be54c170da2cc4b7551d057b78f75', 36, 0), ('concatenate-149be54c170da2cc4b7551d057b78f75', 49, 0), ('concatenate-149be54c170da2cc4b7551d057b78f75', 38, 0), ('concatenate-149be54c170da2cc4b7551d057b78f75', 7, 0), ('concatenate-01a238

## Settings

In [4]:
dirs = {'1 km np': '1km_incorrect_wind_damping',
        '1 km': '1km',
        '4 km': '4km',
        '100 m': 'LES_incorrect_wind_damping'}

perts = {'res': list(dirs.keys()),   # Dataset names.
         'dir': [dirs[x] for x in dirs.keys()],  # Dataset directories.
         'levels': ['850', '730', '600', '500', '412'], # Perturbed levels in hPa.
         'T': ['0.5', '-0.5'],                          # Temperature perturbations in K day-1.
         'q': ['0.0002', '-0.0002']}                    # Specific humidity perturbations in kg kg-1 day-1.

# Input directories with dataset names as keys.
basedir = '/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/'
inputs = wp.input_map(perts=perts, basedir=basedir)
    
# Figure settings.
plt.rcParams['figure.figsize'] = wp.FIGURE_SIZE  # Figure size for non-facetted plots.
plt.rcParams['font.size'] = 12                   # Font size for plots.
plt.rcParams['axes.formatter.useoffset'] = False # Don't use offsets in plots.

# The point at which the RCE run ends and control + perturbation runs begin.
runs_start = {'4 km': '2000-03-01',
              '1 km': '2000-03-01',
              '1 km np': '2000-03-01',
              '100 m':  '2000-04-26'} 

# Start and end times for designated RCE periods.
start_time = {'4 km': '2000-04-01',
              '1 km': '2000-04-01',
              '1 km np': '2000-04-01',
              '100 m':  '2000-05-05'}
end_time =   {'4 km': '2000-06-01',
              '1 km': '2000-06-01',
              '1 km np': '2000-06-01',
              '100 m':  '2000-05-10'}
               
plot_levels = [850, 500, 410, 100] # Pressure levels to plot individually [hPa].

## Read data

The script `~/code/sh/extract_WRF_variables_parallel.sh` runs, in parallel, a python script that extracts variables of interest from WRF `wrfout` files, optionally interpolates the 3D variables to vertical pressure levels, and takes spatial means across horizontal dimensions. These profiles of mean values per time are written to `wrfvar` files, which are then opened here. Note results are cached in `data/WRF`.

In [9]:
reload(wp)
pw_ts, profs = wp.load_cache_data(inputs=inputs, dirs=dirs, runs_start=runs_start, start_time=start_time, end_time=end_time)

Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/control/): Control...
Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_850hPa_T_0.5K/): T 0.5 @850...
Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_850hPa_T_-0.5K/): T -0.5 @850...
Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_850hPa_q_0.0002kgkg-1/): q 0.0002 @850...


Task exception was never retrieved
future: <Task finished name='Task-1232436' coro=<Client._gather.<locals>.wait() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
2024-06-12 23:22:33,911 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_850hPa_q_-0.0002kgkg-1/): q -0.0002 @850...
Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_730hPa_T_0.5K/): T 0.5 @730...


2024-06-12 23:22:47,042 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_730hPa_T_-0.5K/): T -0.5 @730...
Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_730hPa_q_0.0002kgkg-1/): q 0.0002 @730...
Reading 1 km np dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km_incorrect_wind_damping/pert_730hPa_q_-0.0002kgkg-1/): q -0.0002 @730...


...

2024-06-12 23:29:43,387 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


Reading 1 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km/pert_500hPa_q_-0.0002kgkg-1/): q -0.0002 @500...
Reading 1 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km/pert_412hPa_T_0.5K/): T 0.5 @412...
Reading 1 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km/pert_412hPa_T_-0.5K/): T -0.5 @412...
Reading 1 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km/pert_412hPa_q_0.0002kgkg-1/): q 0.0002 @412...
Reading 1 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/1km/pert_412hPa_q_-0.0002kgkg-1/): q -0.0002 @412...


2024-06-12 23:30:18,359 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-06-12 23:32:01,600 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/control/): Control...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_850hPa_T_0.5K/): T 0.5 @850...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_850hPa_T_-0.5K/): T -0.5 @850...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_850hPa_q_0.0002kgkg-1/): q 0.0002 @850...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_850hPa_q_-0.0002kgkg-1/): q -0.0002 @850...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_730hPa_T_0.5K/): T 0.5 @730...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_730hPa_T_-0.5K/): T -0.5 @730...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_730hPa_q_0.0002kgkg-1/): q 0.0002 @730...
Reading 4 km dataset (/g/data/up6/tr2908/em_quarter_ss/v4.1.4/output/4km/pert_730hPa_q_-0.0002kgkg-1/):

KeyError: "['RCE'] not found in axis"

Read MONC data:

In [6]:
monc_cwv = wp.MONC_CWV_data()

## Radiative-convective equilibrium (RCE)

To determine when the simulations have reached RCE, we look for stabilisation of the precipitable water (PW) field. Here is spatially-averaged PW by time for each simulation in WRF. The green highlighted region is the time span over which average profiles are calculated for all runs.

In [ ]:
wp.plot_pw_ts(pw_ts=pw_ts, start_time=start_time, end_time=end_time)

Make a similar plot for the MONC results:

In [ ]:
wp.plot_monc_cwv(monc=monc_cwv)

# Mean profiles over RCE period

In [ ]:
wp.plot_mean_profiles(profs, variables=['tk', 'q', 'ua', 'va', 'rh'], 
                      relabel={'q': 'Water vapor\nmixing ratio\n[kg kg$^{-1}$]', 'ua': 'U wind\n[m s$^{-1}$]',
                               'va': 'V wind\n[m s$^{-1}$]', 'rh': 'Relative\nhumidity [%]'},
                      retick={'tk': [200, 280]})

In [ ]:
wp.plot_mean_profiles(profs, variables=['qcloud', 'qice', 'qsnow', 'qrain', 'qgraup'])

## Perturbation differences in averaged RCE profiles

Differences here are defined as `perturbed - control`. The red vertical line shows zero difference.

In [ ]:
refs = wp.kuang_data(ref_dir='/g/data/up6/tr2908/LRF_SCM_results/')

In [ ]:
monc = wp.MONC_response_data()
monc['model'] = 'MONC'

In [ ]:
# Collect WRF differences together in the same form as the MONC differences.
wrf_profs = xarray.merge([profs[x][['T', 'tk', 'q', 'qcloud', 'qice', 'qsnow', 'qrain', 'qgraup', 'rh']].load().expand_dims({'res': [x]}) for x in profs])

# Convert quantities in kg kg-1 to g kg-1.
for v in ['q', 'qcloud', 'qice', 'qsnow', 'qrain', 'qgraup']:
    wrf_profs[v] = wrf_profs[v]*1000
    
wrf_diffs = wrf_profs.drop_sel(Dataset='Control') - wrf_profs.sel(Dataset='Control')

wrf = wrf_diffs.to_dataframe().reset_index()
wrf['model'] = 'WRF'
wrf = wrf.rename(columns={'Dataset': 'pert', 'level': 'pressure'})

## Collect WRF and MONC differences together.
diffs = pd.concat([wrf, monc]).reset_index(drop=True)
diffs = diffs.sort_values(['pressure', 'model', 'res'], ascending=False)
diffs = diffs.rename(columns={'model': 'Model', 'res': 'Resolution'})

# Give hydrometeors a factor of 1e3
for v in ['q', 'qcloud', 'qice', 'qsnow', 'qrain', 'qgraup']:
    diffs[v] = diffs[v]*1000
    
diffs['pert_group'] = [x.replace('-', '') for x in diffs.pert]
diffs['neg'] = ['-' in x for x in diffs.pert]

pos = diffs[~diffs.neg].copy()
neg = diffs[diffs.neg].copy()

for v in ['T', 'tk', 'q', 'qcloud', 'qice', 'qsnow', 'qrain', 'qgraup', 'rh']:
    neg[v] = -1 * neg[v]
    
diffs = pd.concat([neg, pos])

In [ ]:
# hue_order = ['4 km', '1 km', '500 m', '250 m', '100 m']
hue_order = ['1 km', '1 km np']

perts = [x for x in np.unique(diffs.pert_group) if x[0] == 'T' or x[0] == 'q']
for p in perts:
    p_level = float(p[-3:])
    variables = ['tk', 'rh', 'q', 'qcloud', 'qice', 'qsnow', 'qrain', 'qgraup']
    var_labels = {'tk': 'Temperature\n[K]',
                  'rh': 'RH\n[%]',
                  'q': 'MR Vapour\n[10$^{-3}$ g kg$^{-1}$]',
                  'qcloud': 'MR Cloud\n[10$^{-3}$ g kg$^{-1}$]',
                  'qice': 'MR Ice\n[10$^{-3}$ g kg$^{-1}$]',
                  'qsnow': 'MR Snow\n[10$^{-3}$ g kg$^{-1}$]',
                  'qrain': 'MR Rain\n[10$^{-3}$ g kg$^{-1}$]',
                  'qgraup': 'MR Graupel\n[10$^{-3}$ g kg$^{-1}$]'}
    figsize = (12,8)
    ncols = 4
    nrows = 2
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize, gridspec_kw={'hspace': 0.5})

    assert len(variables) <= ncols * nrows, 'Not enough col/rows.'

    d = diffs[diffs.pert_group == p]
    d = d[d.pressure >= 100]

    for i, variable in enumerate(variables):
        axs.flat[i].axvline(0, color='black')
        axs.flat[i].axhline(p_level, color='black', linestyle='--')

        sns.lineplot(data=d[~d.neg], x=variable, y='pressure', ax=axs.flat[i], style='Model', hue='Resolution', sort=False, estimator=None, legend=(i == ncols-1),
                     hue_order=hue_order[::-1], palette=sns.color_palette("turbo", len(hue_order)), zorder=5)
        
        if len(d[d.neg]) > 0:
            sns.lineplot(data=d[d.neg], x=variable, y='pressure', ax=axs.flat[i], style='Model', hue='Resolution', sort=False, estimator=None,
                         hue_order=hue_order[::-1], palette=sns.color_palette("turbo", len(hue_order)), alpha=0.5, legend=False, zorder=5)
        axs.flat[i].invert_yaxis()

        axs.flat[i].set_ylim(1000, 100)

        # Add Kuang 2010 reference values.
        if variable == 'q':
            r = refs[refs.Dataset == p]
            axs.flat[i].scatter(r.q * 1000, r.level, facecolors='none', edgecolors='black', zorder=10, s=30)
        if variable == 'tk':
            r = refs[refs.Dataset == p]
            axs.flat[i].scatter(r.tk, r.level, facecolors='none', edgecolors='black', zorder=10, s=30)
        
        if variable in var_labels:
            axs.flat[i].set_xlabel(var_labels[variable])

        if i % ncols == 0:
            axs.flat[i].set_ylabel('Pressure [hPa]')
        else: 
            axs.flat[i].set_ylabel('')
            axs.flat[i].set_yticks([])

    sns.move_legend(axs.flat[ncols-1], 'upper left', bbox_to_anchor=(1, 1))
    _ = plt.suptitle(p, y=0.93)
    plt.savefig(f'paper/figures/pert_diffs_{p.replace(" ", "_")}.pdf', bbox_inches='tight')
    plt.show()
    plt.close()